# 영업 성공 여부 분류 경진대회

# 할 것

결측치 이상치 처리
파생변수 생성

## 1. 데이터 확인

### 필수 라이브러리

In [21]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def seed_everything(seed):
    np.random.seed(seed) #numpy를 사용할 경우 고정
seed_everything(42)

### 데이터 셋 읽어오기

In [22]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [23]:
df_train[:10]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,product_subcategory,product_modelname,customer_country.1,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.00,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,NaN,purchasing,62,Quotation or purchase consultation,multi-split,NaN,NaN,/Quezon City/Philippines,entry level,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.00,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,NaN,media and communication,96,Quotation or purchase consultation,multi-split,NaN,NaN,/PH-00/Philippines,ceo/founder,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.00,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,NaN,engineering,56,Product Information,single-split,NaN,NaN,/Kolkata /India,partner,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.00,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,NaN,entrepreneurship,44,Quotation or purchase consultation,vrf,NaN,NaN,/Bhubaneswar/India,ceo/founder,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.00,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,NaN,consulting,97,Quotation or purchase consultation,multi-split,NaN,NaN,/Hyderabad/India,partner,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True
5,1.00,/Abuja/Nigeria,AS,0.040816,16328,End-Customer,SMB,NaN,NaN,NaN,NaN,program and project management,1114,Quotation or purchase consultation,chiller,NaN,NaN,/Abuja/Nigeria,manager,LGEAF,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,5,True
6,0.75,"/Jeddah, KSA/Saudi Arabia",AS,0.040816,20664,End-Customer,SMB,NaN,NaN,NaN,NaN,engineering,420,Quotation or purchase consultation,single-split,NaN,NaN,"/Jeddah, KSA/Saudi Arabia",manager,LGESJ,NaN,1,0,0.003079,0.026846,corporate / office,Engineering,6,True
7,1.00,/Guwahati/India,AS,0.088889,17983,End-Customer,SMB,NaN,NaN,NaN,NaN,sales,205,Quotation or purchase consultation,vrf,NaN,NaN,/Guwahati/India,partner,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Manufacturing,7,True
8,0.75,/Cebu city/Philippines,AS,0.066667,30867,Specifier/ Influencer,Enterprise,3.0,NaN,NaN,NaN,other,103,Quotation or purchase consultation,multi-split,NaN,NaN,/Cebu city/Philippines,vice president,LGEPH,less than 3 months,0,0,0.003079,0.026846,corporate / office,Construction,8,True
9,0.75,"/hauz khas,delhi/India",AS,0.088889,6084,End-Customer,SMB,NaN,NaN,NaN,NaN,other,252,Quotation or purchase consultation,vrf,NaN,NaN,"/hauz khas,delhi/India",manager,LGEIL,3 months ~ 6 months,1,0,0.003079,0.026846,corporate / office,NaN,9,True


In [24]:
df_train['business_unit'].unique()

array(['AS', 'ID', 'IT', 'Solution', 'CM'], dtype=object)

In [25]:
# 결측치가 있는 변수 및 결측률 파악 

mis_val = df_train.isnull().sum()
mis_val_bool = mis_val >= 1
mis_val_df = pd.concat([mis_val, mis_val_bool], axis = 1)
mis_val_df = mis_val_df.rename(columns = {0 : 'mis_val', 1 : 'mis_val_bool'})

mis_val_data = mis_val_df.loc[mis_val_df['mis_val_bool'] == True, :]
mis_val_data['ratio'] = mis_val_data['mis_val'] / 59299
mis_val_data_sorted = mis_val_data.sort_values(by='ratio', ascending=True)
mis_val_data_sorted

/tmp/ipykernel_205/929980573.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mis_val_data['ratio'] = mis_val_data['mis_val'] / 59299


,mis_val,mis_val_bool,ratio
inquiry_type,941,True,0.015869
customer_country.1,982,True,0.016560
customer_country,982,True,0.016560
customer_job,18733,True,0.315908
product_category,19374,True,0.326717
expected_timeline,30863,True,0.520464
business_area,40882,True,0.689421
ver_win_rate_x,40882,True,0.689421
customer_type,43961,True,0.741345
ver_win_ratio_per_bu,43995,True,0.741918


In [26]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   customer_country         58317 non-null  object 
 2   business_unit            59299 non-null  object 
 3   com_reg_ver_win_rate     14568 non-null  float64
 4   customer_idx             59299 non-null  int64  
 5   customer_type            15338 non-null  object 
 6   enterprise               59299 non-null  object 
 7   historical_existing_cnt  13756 non-null  float64
 8   id_strategic_ver         3444 non-null   float64
 9   it_strategic_ver         1121 non-null   float64
 10  idit_strategic_ver       4565 non-null   float64
 11  customer_job             40566 non-null  object 
 12  lead_desc_length         59299 non-null  int64  
 13  inquiry_type             58358 non-null  object 
 14  product_category      

### 전처리
inquiry_type,customer_country - 대소문자 통합

customer_country 결측치 행 삭제

In [27]:
# customer_country , customer_country.1의 값이 일치함.
pre_train = df_train.drop(labels = ['customer_country.1'],axis=1)
pre_test = df_test.drop(labels = ['customer_country.1','id'],axis=1)

dtypes = pre_train.dtypes

# 각 열의 고유값 개수 확인
unique_value_counts = pre_train.nunique()

# 수치형 변수가 아닌 컬럼을 저장할 리스트 초기화
object_columns = []

# 각 열에 대해 데이터 타입이 'object'이거나 고유값 개수가 일정 이하인 경우를 확인하여 non_numeric_columns 리스트에 추가
for col in pre_train.columns:
    if dtypes[col] == 'object':
        object_columns.append(col)

# 수치형 변수가 아닌 컬럼 출력
print("범주형 컬럼:", object_columns,"\n")
print("수치형 컬럼 :", list(set(pre_train.columns)-set(object_columns)))

범주형 컬럼: ['customer_country', 'business_unit', 'customer_type', 'enterprise', 'customer_job', 'inquiry_type', 'product_category', 'product_subcategory', 'product_modelname', 'customer_position', 'response_corporate', 'expected_timeline', 'business_area', 'business_subarea'] 

수치형 컬럼 : ['id_strategic_ver', 'ver_pro', 'customer_idx', 'bant_submit', 'ver_win_rate_x', 'it_strategic_ver', 'historical_existing_cnt', 'com_reg_ver_win_rate', 'is_converted', 'ver_cus', 'lead_desc_length', 'ver_win_ratio_per_bu', 'lead_owner', 'idit_strategic_ver']


In [28]:
# 각 컬럼별로 이상치 판별 임계값을 계산하는 함수
def find_outlier_thresholds(data, threshold=3):
    thresholds = {}
    min_outliers = {}  # 각 컬럼별 최소 이상치 값을 저장하기 위한 딕셔너리
    
    num_columns = ['com_reg_ver_win_rate', 'ver_pro', 'ver_win_ratio_per_bu', 
                   'idit_strategic_ver', 'lead_owner', 'customer_idx', 'bant_submit', 
                   'id_strategic_ver', 'is_converted', 'historical_existing_cnt', 'it_strategic_ver', 
                   'ver_win_rate_x', 'lead_desc_length', 'ver_cus']
    
    for column in num_columns:
        # 각 컬럼의 Z-Score 계산
        z_scores = (data[column] - data[column].mean()) / data[column].std()
        
        # Z-Score가 임계값보다 큰 데이터를 이상치로 판단
        outliers = data[np.abs(z_scores) > threshold]
        
        # 이상치 판별 임계값 저장
        thresholds[column] = {
            'threshold': threshold,
            'outliers_count': len(outliers),
            'outliers_indices': list(outliers.index)
        }
        
        # 최소 이상치 값 찾기
        if len(outliers) > 0:
            min_outliers[column] = outliers[column].min()
    
    return thresholds, min_outliers

# 예시 데이터 프레임 생성 (스트링 데이터를 가진 'C' 컬럼 추가)

# 이상치 판별 임계값 계산 (임계값은 3으로 설정)
outlier_thresholds, min_outliers = find_outlier_thresholds(pre_train, threshold=3)

# Y_LABEL 값에 따라 이상치 개수와 각 컬럼별 최소 이상치 값을 구분하여 출력
for column, threshold_info in outlier_thresholds.items():
    print(f"Column: {column}")
    print(f"Threshold: {threshold_info['threshold']}")
    
    outliers_true = len([idx for idx in threshold_info['outliers_indices'] if pre_train.loc[idx, 'is_converted'] == True])
    print(f"Number of Outliers : {outliers_true}")
    outliers_false = len([idx for idx in threshold_info['outliers_indices'] if pre_train.loc[idx, 'is_converted'] == False])
    print(f"Number of Outliers : {outliers_false}")
    
    
    # 최소 이상치 값 출력
    if column in min_outliers:
        print(f"Min Outlier Value: {min_outliers[column]}")

    print()


Column: com_reg_ver_win_rate
Threshold: 3
Number of Outliers : 340
Number of Outliers : 493
Min Outlier Value: 0.6153846153846154

Column: ver_pro
Threshold: 3
Number of Outliers : 278
Number of Outliers : 2735
Min Outlier Value: 1

Column: ver_win_ratio_per_bu
Threshold: 3
Number of Outliers : 20
Number of Outliers : 85
Min Outlier Value: 0.2272727272727272

Column: idit_strategic_ver
Threshold: 3
Number of Outliers : 0
Number of Outliers : 0

Column: lead_owner
Threshold: 3
Number of Outliers : 96
Number of Outliers : 433
Min Outlier Value: 958

Column: customer_idx
Threshold: 3
Number of Outliers : 0
Number of Outliers : 0

Column: bant_submit
Threshold: 3
Number of Outliers : 0
Number of Outliers : 0

Column: id_strategic_ver
Threshold: 3
Number of Outliers : 0
Number of Outliers : 0

Column: is_converted
Threshold: 3
Number of Outliers : 4850
Number of Outliers : 0
Min Outlier Value: True

Column: historical_existing_cnt
Threshold: 3
Number of Outliers : 22
Number of Outliers : 43

In [29]:
"""
범주형 데이터에 넣을 것들 후보
'it_strategic_ver' 'id_strategic_ver' 'idit_strategic_ver' -> array([nan,  1.])
"""
columns_categorized = ['it_strategic_ver','id_strategic_ver','idit_strategic_ver'] + ['lead_owner','ver_cus','ver_pro','customer_idx']

for n in columns_categorized:
    pre_train[n] = pre_train[n].fillna(0)
    pre_train[n] = pre_train[n].astype('int').astype('object')

In [54]:
object_columns

['customer_country',
 'business_unit',
 'customer_type',
 'enterprise',
 'customer_job',
 'inquiry_type',
 'product_category',
 'product_subcategory',
 'product_modelname',
 'customer_position',
 'response_corporate',
 'expected_timeline',
 'business_area',
 'business_subarea',
 'it_strategic_ver',
 'id_strategic_ver',
 'idit_strategic_ver']

In [30]:
pre_train['it_strategic_ver'].head()

0    0
1    0
2    0
3    0
4    0
Name: it_strategic_ver, dtype: object

In [31]:
def preprocess_1(col):
    pre_train[col] = pre_train[col].str.lower()
    pre_test[col] = pre_test[col].str.lower()
    
    mask_train = pre_train[col].notnull()
    mask_test = pre_test[col].notnull()
    pre_train.loc[mask_train, col] = pre_train.loc[mask_train, col].apply(lambda x: x.replace(" ", ""))
    pre_test.loc[mask_test, col] = pre_test.loc[mask_test, col].apply(lambda x: x.replace(" ", ""))    

In [32]:
for n in object_columns:
    preprocess_1(n)
    
#test data에 customer_country = nan 이 없어서 행삭제 해도 될듯. 삭제양도 많지 않음.
pre_train = pre_train.dropna(subset=['customer_country'])

object_columns = object_columns + ['it_strategic_ver','id_strategic_ver','idit_strategic_ver','lead_owner','ver_cus','ver_pro','customer_idx']
numerical_columns = list(set(pre_train.columns)-set(object_columns))

# 보류
# """
# df_train['customer_country'] = df_train['customer_country'].apply(lambda x: x.split('/')[-1])
# df_test['customer_country'] = df_test['customer_country'].apply(lambda x: x.split('/')[-1])
# """

### 레이블 인코딩

In [ ]:
# def label_encoding(series: pd.Series) -> pd.Series:
#     """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

#     my_dict = {}

#     # 모든 요소를 문자열로 변환
#     series = series.astype(str)

#     for idx, value in enumerate(sorted(series.unique())):
#         my_dict[value] = idx
#     series = series.map(my_dict)

#     return series

In [ ]:
# # 레이블 인코딩할 칼럼들
# label_columns = [
#     "customer_country",
#     "business_subarea",
#     "business_area",
#     "business_unit",
#     "customer_type",
#     "enterprise",
#     "customer_job",
#     "inquiry_type",
#     "product_category",
#     "product_subcategory",
#     "product_modelname",
#     "customer_position",
#     "response_corporate",
#     "expected_timeline",
# ]

# df_all = pd.concat([pre_train[label_columns], pre_test[label_columns]])

# for col in label_columns:
#     df_all[col] = label_encoding(df_all[col])

# for col in label_columns:  
#     pre_train[col] = df_all.iloc[: len(pre_train)][col]
#     pre_test[col] = df_all.iloc[len(pre_train):][col]

In [ ]:
# len(pre_train.info())

## 특징추출

In [ ]:
# import catboost
# from catboost import CatBoostClassifier
# from sklearn.model_selection import StratifiedKFold

# stf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)

# result = []

# #y_label을 제외한 컬럼
# x_columns = list(set(pre_train.columns)-set('is_converted'))

# train_x = pre_train.drop(columns=['is_converted'])
# train_y = pre_train['is_converted']

# n = 0

# for train_index, valid_index in stf.split(train_x, train_y):
#     train_X = np.array(train_x)
#     train_Y = np.array(train_y)

#     X_train, X_valid = train_X[train_index], train_X[valid_index]
#     y_train, y_valid = train_Y[train_index], train_Y[valid_index]

#     model = CatBoostClassifier(iterations = 1000, random_seed=42,
#                                verbose=0)

#     summary = model.select_features(
#         X = X_train,
#         y = y_train,
#         features_for_select = '0-26',
#         eval_set=(X_valid, y_valid),
#         num_features_to_select=20,
#         steps=13,
#         train_final_model=False,
#         logging_level='Silent',
#         plot=False
#     )

#     result.append(summary['selected_features'])
#     print(n)
#     n+=1

# feature_selected = []
# for i in range(5):
#     for j in result[i]:
#         feature_selected.append(j)
        
# print('done')

In [ ]:
# # feature_selected = [df_train.columns[i] for i in feature_selected]
# feature_selected = list(set(feature_selected))
# feature_nonSelected = list(set(train_x.columns) - set(feature_selected))

# print('선택된 columns : ',len(feature_selected),'\n',feature_selected,'\n\n')
# print('선택되지 않은 columns : ',len(feature_nonSelected),'\n',feature_nonSelected)

In [ ]:
# feature_selected = 

# feature_selected_index = [pre_train.columns.get_loc(col) for col in feature_selected]

# pre_train = pre_train[feature_selected]
# pre_test = pre_test[feature_selected]

# t = pre_train.copy()
# corr = t.corr()
# corr.style.background_gradient(cmap = 'coolwarm')

In [ ]:
# print(len(pre_train.columns),'\n',pre_train.columns)
# print('\n\n\n')
# print(len(pre_test.columns),'\n',pre_test.columns)

## 3. 모델 학습

### 모델 정의 

In [33]:
pre_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58317 entries, 0 to 59298
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              58317 non-null  float64
 1   customer_country         58317 non-null  object 
 2   business_unit            58317 non-null  object 
 3   com_reg_ver_win_rate     13927 non-null  float64
 4   customer_idx             58317 non-null  object 
 5   customer_type            15310 non-null  object 
 6   enterprise               58317 non-null  object 
 7   historical_existing_cnt  13615 non-null  float64
 8   id_strategic_ver         58317 non-null  object 
 9   it_strategic_ver         58317 non-null  object 
 10  idit_strategic_ver       58317 non-null  object 
 11  customer_job             40133 non-null  object 
 12  lead_desc_length         58317 non-null  int64  
 13  inquiry_type             57948 non-null  object 
 14  product_category         39

In [46]:
y_valid.type()

AttributeError: 'numpy.ndarray' object has no attribute 'type'

### 모델 학습

In [47]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

# Stratified K-Fold 교차 검증 설정
n_splits = 5  # 원하는 Fold 수
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# 최적의 하이퍼파라미터 설정
best_params = {'depth': 10, 'iterations': 1500, 'learning_rate': 0.01}

# 교차 검증 수행 및 평가 지표 저장
macro_f1_scores = []
probabilities_list = []
best_f1_score = 0

train_x = pre_train.drop(columns=['is_converted'])
train_y = pre_train['is_converted']
train_x = train_x.fillna(0)

X = train_x.values
y = train_y.values

for train_index, valid_index in skf.split(X, y):
    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]

    # CatBoost 모델 생성 및 학습
    model = CatBoostClassifier(loss_function='Logloss', cat_features=np.where(pre_train.dtypes == np.object)[0], eval_metric='F1', 
                               random_seed=42, verbose=500, **best_params)
    
    model.fit(X_train, y_train,)

    # 검증 데이터에 대한 예측 및 평가 (Macro F1 Score)
    valid_predictions = model.predict(X_valid)
    valid_predictions = np.array([True if pred == 'True' else False for pred in valid_predictions])
    print(valid_predictions,'\n\n\n')
    print(y_valid)
    
    macro_f1 = f1_score(y_valid, valid_predictions, average='macro')
    macro_f1_scores.append(macro_f1)

    if macro_f1 > best_f1_score:
        best_f1_score = macro_f1
        best_classification_model = model

# Macro F1 Score 출력
print("Macro F1 Scores:", macro_f1_scores)


/tmp/ipykernel_205/1889840750.py:30: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  model = CatBoostClassifier(loss_function='Logloss', cat_features=np.where(pre_train.dtypes == np.object)[0], eval_metric='F1',


0:	learn: 0.2607004	total: 145ms	remaining: 3m 37s
500:	learn: 0.8650631	total: 1m 23s	remaining: 2m 46s
1000:	learn: 0.9018203	total: 2m 52s	remaining: 1m 25s
1499:	learn: 0.9248978	total: 4m 18s	remaining: 0us
[ True  True False ... False False False] 



[ True  True  True ... False False False]


/tmp/ipykernel_205/1889840750.py:30: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  model = CatBoostClassifier(loss_function='Logloss', cat_features=np.where(pre_train.dtypes == np.object)[0], eval_metric='F1',


0:	learn: 0.4824609	total: 184ms	remaining: 4m 36s
500:	learn: 0.8645389	total: 1m 22s	remaining: 2m 44s
1000:	learn: 0.9020057	total: 2m 50s	remaining: 1m 25s
1499:	learn: 0.9255289	total: 4m 17s	remaining: 0us
[False False False ... False False False] 



[ True  True False ... False False False]


/tmp/ipykernel_205/1889840750.py:30: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  model = CatBoostClassifier(loss_function='Logloss', cat_features=np.where(pre_train.dtypes == np.object)[0], eval_metric='F1',


0:	learn: 0.4895085	total: 42.1ms	remaining: 1m 3s
500:	learn: 0.8607296	total: 1m 21s	remaining: 2m 42s
1000:	learn: 0.8997126	total: 2m 47s	remaining: 1m 23s
1499:	learn: 0.9202714	total: 4m 12s	remaining: 0us
[ True  True False ... False False False] 



[ True  True  True ... False False False]


/tmp/ipykernel_205/1889840750.py:30: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  model = CatBoostClassifier(loss_function='Logloss', cat_features=np.where(pre_train.dtypes == np.object)[0], eval_metric='F1',


0:	learn: 0.4146725	total: 200ms	remaining: 4m 59s
500:	learn: 0.8691438	total: 1m 22s	remaining: 2m 43s
1000:	learn: 0.9058874	total: 2m 50s	remaining: 1m 25s
1499:	learn: 0.9254448	total: 4m 19s	remaining: 0us
[ True  True False ... False False False] 



[ True  True  True ... False False False]


/tmp/ipykernel_205/1889840750.py:30: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  model = CatBoostClassifier(loss_function='Logloss', cat_features=np.where(pre_train.dtypes == np.object)[0], eval_metric='F1',


0:	learn: 0.2721919	total: 48.1ms	remaining: 1m 12s
500:	learn: 0.8605472	total: 1m 23s	remaining: 2m 46s
1000:	learn: 0.8938805	total: 2m 50s	remaining: 1m 25s
1499:	learn: 0.9177771	total: 4m 17s	remaining: 0us
[False  True False ... False False False] 



[ True  True  True ... False False False]
Macro F1 Scores: [0.9280228986979451, 0.9250464895779114, 0.9313136131613082, 0.9225421149206559, 0.927839555355455]


## 4. 제출하기

### 테스트 데이터 예측

In [55]:
for n in columns_categorized:
    pre_test[n] = pre_test[n].fillna(0)
    pre_test[n] = pre_test[n].astype('int').astype('object')

In [56]:
# 예측에 필요한 데이터 분리
x_test = pre_test.drop(["is_converted"], axis=1)
x_test.head()

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,product_subcategory,product_modelname,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner
0,0.00,//brazil,id,0.073248,47466,endcustomer,enterprise,53.0,0,0,0,consulting,3,NaN,NaN,NaN,NaN,none,lgesp,NaN,1,0,0.001183,0.049840,retail,electronics&telco,278
1,0.25,400nstateoffranklinrdcloudit/johnsoncity/unite...,it,NaN,5405,endcustomer,smb,NaN,0,0,0,NaN,23,quotationorpurchaseconsultation,NaN,NaN,NaN,none,lgeus,NaN,0,0,0.000013,NaN,transportation,others,437
2,1.00,//u.a.e,id,NaN,13597,specifier/influencer,smb,NaN,0,0,0,informationtechnology,138,quotationorpurchaseconsultation,hospitaltv,ut662mseries,50ut662m(mea),manager,lgegf,lessthan3months,0,0,0.000060,0.131148,hospital&healthcare,generalhospital,874
3,0.50,/madison/unitedstates,id,0.118644,17204,NaN,enterprise,NaN,0,0,0,sales,3,quotationorpurchaseconsultation,NaN,digitalsignageorcommercialtvs,NaN,none,lgeus,morethanayear,0,0,0.001183,0.049840,retail,NaN,194
4,1.00,/saopaulo/brazil,id,0.074949,2329,endcustomer,enterprise,2.0,1,0,1,engineering,107,quotationorpurchaseconsultation,ledsignage,lgledbloc,lsaa012,others,lgesp,lessthan3months,1,1,0.003079,0.064566,corporate/office,engineering,167


In [50]:
train_x.head()

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,product_subcategory,product_modelname,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner
0,1.0,/quezoncity/philippines,as,0.066667,32160,end-customer,enterprise,0.0,0,0,0,purchasing,62,quotationorpurchaseconsultation,multi-split,0,0,entrylevel,lgeph,lessthan3months,1,0,0.003079,0.026846,corporate/office,engineering,0
1,1.0,/ph-00/philippines,as,0.066667,23122,end-customer,enterprise,12.0,0,0,0,mediaandcommunication,96,quotationorpurchaseconsultation,multi-split,0,0,ceo/founder,lgeph,lessthan3months,1,0,0.003079,0.026846,corporate/office,advertising,1
2,1.0,/kolkata/india,as,0.088889,1755,end-customer,enterprise,144.0,0,0,0,engineering,56,productinformation,single-split,0,0,partner,lgeil,lessthan3months,1,0,0.003079,0.026846,corporate/office,construction,2
3,1.0,/bhubaneswar/india,as,0.088889,4919,end-customer,enterprise,0.0,0,0,0,entrepreneurship,44,quotationorpurchaseconsultation,vrf,0,0,ceo/founder,lgeil,lessthan3months,1,0,0.003079,0.026846,corporate/office,it/software,3
4,1.0,/hyderabad/india,as,0.088889,17126,specifier/influencer,enterprise,0.0,0,0,0,consulting,97,quotationorpurchaseconsultation,multi-split,0,0,partner,lgeil,lessthan3months,0,0,0.003079,0.026846,corporate/office,0,4


In [57]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5271 entries, 0 to 5270
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              5271 non-null   float64
 1   customer_country         5271 non-null   object 
 2   business_unit            5271 non-null   object 
 3   com_reg_ver_win_rate     1788 non-null   float64
 4   customer_idx             5271 non-null   object 
 5   customer_type            3814 non-null   object 
 6   enterprise               5271 non-null   object 
 7   historical_existing_cnt  1275 non-null   float64
 8   id_strategic_ver         5271 non-null   object 
 9   it_strategic_ver         5271 non-null   object 
 10  idit_strategic_ver       5271 non-null   object 
 11  customer_job             3832 non-null   object 
 12  lead_desc_length         5271 non-null   int64  
 13  inquiry_type             3979 non-null   object 
 14  product_category        

In [58]:
test_pred = model.predict(x_test.fillna(0))
print(test_pred)
sum(int(pred == 'True') for pred in test_pred)

['True' 'True' 'False' ... 'False' 'False' 'True']


678

### 제출 파일 작성

In [59]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**